In [1]:
! pip install sentence-splitter

     |████████████████████████████████| 44 kB 1.4 MB/s 


In [2]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 5.6 MB/s 
     |████████████████████████████████| 895 kB 18.7 MB/s 
     |████████████████████████████████| 596 kB 51.1 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 41.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
! pip install SentencePiece

     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [4]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [14]:
context = "Papersdrop, a place to learn, create and share thoughts and voices of your own. A person can create amazing content with just a splash of ink. We at Papersdrop want to serve as a medium for you to write. With us, your opinion matters and we provide you with various seo tools and features to write beautiful and engaging content. With every drop of ink, we give you a platform to create content like never before."
print(context)

Papersdrop, a place to learn, create and share thoughts and voices of your own. A person can create amazing content with just a splash of ink. We at Papersdrop want to serve as a medium for you to write. With us, your opinion matters and we provide you with various seo tools and features to write beautiful and engaging content. With every drop of ink, we give you a platform to create content like never before.


In [15]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

['Papersdrop, a place to learn, create and share thoughts and voices of your own.',
 'A person can create amazing content with just a splash of ink.',
 'We at Papersdrop want to serve as a medium for you to write.',
 'With us, your opinion matters and we provide you with various seo tools and features to write beautiful and engaging content.',
 'With every drop of ink, we give you a platform to create content like never before.']

In [16]:
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [17]:
paraphrase

[['Papersdrop is a place to learn, create and share.'],
 ['A person can make amazing content with a small amount of ink.'],
 ['Papersdrop wants to serve as a medium for you to write.'],
 ['We provide you with various tools and features to write beautiful and engaging content.'],
 ['With every drop of ink, we give you a platform to create.']]

In [18]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

['Papersdrop is a place to learn, create and share.',
 'A person can make amazing content with a small amount of ink.',
 'Papersdrop wants to serve as a medium for you to write.',
 'We provide you with various tools and features to write beautiful and engaging content.',
 'With every drop of ink, we give you a platform to create.']

In [19]:
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'Papersdrop is a place to learn, create and share. A person can make amazing content with a small amount of ink. Papersdrop wants to serve as a medium for you to write. We provide you with various tools and features to write beautiful and engaging content. With every drop of ink, we give you a platform to create.'